<a href="https://colab.research.google.com/github/inkiga/dw_matrix_road_sign/blob/master/matrix_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size',128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model

def predict(model_trained, X_test, y_test, scoring = accuracy_score):

  y_test_norm = np.argmax(y_test, axis = 1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis =1)

  return accuracy_score(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters = 32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
  Conv2D(filters = 32, kernel_size=(3,3), activation='relu',padding='same'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters = 64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters = 64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),
  
  Flatten(),
  
  Dense(1024, activation = 'relu'),
  Dropout(0.3),

  Dense(1024, activation = 'relu'),
  Dropout(0.3),
  
  Dense(num_classes, activation = 'softmax'),            
])


In [0]:
model = get_cnn_v5(input_shape, num_classes) 
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.3579 - accuracy: 0.3362 - val_loss: 0.7638 - val_accuracy: 0.7646
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 0.6543 - accuracy: 0.7903 - val_loss: 0.1481 - val_accuracy: 0.9591
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3076 - accuracy: 0.9042 - val_loss: 0.0846 - val_accuracy: 0.9759
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2048 - accuracy: 0.9395 - val_loss: 0.0352 - val_accuracy: 0.9898
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.1618 - accuracy: 0.9531 - val_loss: 0.0324 - val_accuracy: 0.9911


0.9684807256235828

In [0]:
def func_obj(params):

  model = get_cnn_v5(input_shape, num_classes)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size',128)),
      epochs = 7,
      verbose = 0,
  )
  
  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials(),
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.3378 - accuracy: 0.9062

 12/138 [=>............................]
 - ETA: 0s - loss: 0.0640 - accuracy: 0.9818

 24/138 [====>.........................]
 - ETA: 0s - loss: 0.1852 - accuracy: 0.9453

 37/138 [=======>......................]
 - ETA: 0s - loss: 0.1249 - accuracy: 0.9628

 49/138 [=========>....................]
 - ETA: 0s - loss: 0.1237 - accuracy: 0.9643

 62/138 [============>.................]
 - ETA: 0s - loss: 0.1579 - accuracy: 0.9597


In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters = 32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
    Conv2D(filters = 32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),
    
    Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters = 64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),
    
    Conv2D(filters = 128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters = 128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),
    
    Flatten(),
    
    Dense(1024, activation = 'relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation = 'relu'),
    Dropout(params['dropout_dense_block_two']),
    
    Dense(num_classes, activation = 'softmax'),            
  ])

In [0]:
def func_obj(params):

  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size',128)),
      epochs = 5,
      verbose = 0,
  )
  
  score = model.evaluate(X_test, y_test, verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 130.0, 'dropout_cnn_block_one': 0.31461250060588536, 'dropout_cnn_block_three': 0.45689920463570993, 'dropout_cnn_block_two': 0.48180392463977356, 'dropout_dense_block_one': 0.4475642221122457, 'dropout_dense_block_two': 0.316625182051313}
accuracy=0.9478458166122437
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.4363661169102373, 'dropout_cnn_block_three': 0.41272279172507453, 'dropout_cnn_block_two': 0.49103814303174764, 'dropout_dense_block_one': 0.45865047469791825, 'dropout_dense_block_two': 0.30949994231377187}
accuracy=0.934920608997345
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.41013708359754, 'dropout_cnn_block_three': 0.34769595895806654, 'dropout_cnn_block_two': 0.49982623139687105, 'dropout_dense_block_one': 0.6078454375510907, 'dropout_dense_block_two': 0.5462198578957219}
accuracy=0.8891156315803528
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.390032537772953, 'dropout_cnn_block_three': 0.4582452005107333, 'dropout_cnn_block_two': 0.394463371631